<a href="https://colab.research.google.com/github/sonthuybacha/100-Days-Of-ML-Code/blob/master/geemap_cartoee_global_age_old_young_ratio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook uses [geemap](https://geemap.org/) with [cartoee](https://cartoee.readthedocs.io/en/latest/introduction.html) to make a figure of global
age ratio of old persons (>= 65) to young persons (20-49) using the [Gridded Population of the World Version 4.11](https://developers.google.com/earth-engine/datasets/catalog/CIESIN_GPWv411_GPW_Basic_Demographic_Characteristics) Earth Engine dataset.

See these notebooks for other geemap/cartoee examples:

- https://github.com/giswqs/geemap/blob/master/examples/notebooks/cartoee_colab.ipynb
- https://github.com/giswqs/geemap/blob/master/examples/notebooks/cartoee_projections.ipynb
- https://github.com/giswqs/geemap/blob/master/examples/notebooks/cartoee_quickstart.ipynb

In [ ]:
# Only run the install once, after install then restart session and proceed.

# Install the Proj and GEOS libraries.
!apt-get install libproj-dev proj-bin
!apt-get install libgeos-dev

# Install cartopy and geemap with all of the dependencies prebuilt.
!pip install cartopy geemap

In [ ]:
# Import packages.
import matplotlib.patches as mpatches
import ee
import geemap.eefolium as geemap
from geemap import cartoee
import cartopy.crs as ccrs

%pylab inline
Map = geemap.Map()

In [ ]:
# Import GPW demographics dataset.
gpw_demog = (ee.ImageCollection('CIESIN/GPWv411/GPW_Basic_Demographic_Characteristics')
    .select('basic_demographic_characteristics'))

# Get "young" population; filter 20 - 39 years old and sum.
young = (gpw_demog.filter(ee.Filter.expression(
    'Age_Group == "20-24" || ' +
    'Age_Group == "25-29" || ' +
    'Age_Group == "30-34" || ' +
    'Age_Group == "35-39"'))
    .filter(ee.Filter.expression('Sex == "b"'))
    .sum()
    .selfMask())

# Get "old" population; filter >= 65 years old.
old = (gpw_demog.filter(ee.Filter.expression(
    'Age_Group == "65 - "'))
    .filter(ee.Filter.expression('Sex == "b"'))
    .first()
    .selfMask())

# Calculate ratio of old persons to young persons and apply vis params.
old_young_ratio_vis = {
  'min': 0,
  'max': 2,
  'palette': '#5ab4ac,#ffffff,#d8b365'
}
old_young_ratio = old.divide(young).visualize(**old_young_ratio_vis);

# Prepare land/water layer.
mask_fill = ee.Image(1).visualize(**{'palette': '#B0B0B0'});
land_water = (ee.Image('MODIS/MOD44W/MOD44W_005_2000_02_24')
    .select('water_mask')
    .visualize(**{'palette': '#282828,#B0B0B0', 'min': 0, 'max': 1}))
land_water = mask_fill.blend(land_water)

# Blend land/water with old-young ratio.
final_img = land_water.blend(old_young_ratio)

# Set figure size and background color.
fig = plt.figure(figsize=(15,9.5), facecolor='#ffffff')

# Create a new Interrupted Goode Homolosine projection.
projection = ccrs.InterruptedGoodeHomolosine()

# Define bounding box to request data.
region = [-180, -90, 180, 90]

# Plot the result with cartoee using Interrupted Goode Homolosine projection.
ax = cartoee.get_map(final_img, region=region, proj=projection)

# Add a colorbar.
cb = cartoee.add_colorbar(
    ax, loc='right', vis_params=old_young_ratio_vis,
    label='Population age ratio (old/young)')

plt.show()